## Using an SVM Classification model for Dow-like index


### Setup

* Index Stocks (DJIA-like) daily open, high, low, and close collected for the list of stocks below,  for 3 years
* AAPL,AXP,BA,CAT,CSCO,CVX,DIS,GE,GOOG,HD,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PFE,PG,RTN,UTX,WFC

* Depending on the model, the number of features selected could be:

* ema(4)                     float64
* ema(10)                    float64
* macd                       float64
* macd 1st d                 float64
* high low span              float64
* high above ema             float64
* close above ema            float64
* close relative to high     float64
* volume change              float64
* volume relative to span    float64
* 3 days ago                   int64
* 2 days ago                   int64
* 1 day ago                    int64
* close relative to low      float64
* open less than close         int64

* The target is a signal "to buy"  int64,  which intends to signal when the right time is to buy the stock

    
    
    


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
#price_data = pd.read_csv('/Final_Project/Model Generators/Input Data/Day Data/day_data_csv.csv')
price_data = pd.read_csv('day data complete.csv')
price_data.head()

,id,stock,close,open,high,low,volume,date,ema(4),ema(10),...,close relative to high,volume change,volume relative to span,3 days ago,2 days ago,1 day ago,close relative to low,open less than close,central pivot point,to buy
0,772,AAPL,93.64,93.965,94.08,92.40,48160104,5/2/2016,1.022028,1.218713,...,0.004699,-0.422993,-0.042415,1,0,0,-0.013242,0,-0.002848,0
1,771,AAPL,95.18,94.200,95.74,93.68,56831277,5/3/2016,1.003295,0.969274,...,0.005884,0.152577,0.141851,0,0,0,-0.015760,0,-0.003292,0
2,770,AAPL,94.19,95.200,95.90,93.82,41025475,5/4/2016,1.008304,1.017578,...,0.018155,-0.385268,-0.057319,0,0,1,-0.003928,1,0.004742,0
3,769,AAPL,93.24,94.000,94.07,92.68,35890500,5/5/2016,1.011146,1.013491,...,0.008902,-0.143073,-0.104197,0,1,0,-0.006006,0,0.000965,0
4,768,AAPL,92.72,93.370,93.45,91.85,43699886,5/6/2016,1.010090,1.013327,...,0.007873,0.178705,0.096563,1,0,0,-0.009383,0,-0.000503,0


In [3]:
# Transform the data 

# 1. Select the Features you want (in columns)


data_raw = price_data[['ema(4)', 'ema(10)', 'macd', 'macd 1st d', 'high low span',
       'high above ema', 'close above ema', 'close relative to high',
       'volume change', 'volume relative to span', '3 days ago', '2 days ago',
       '1 day ago', 'close relative to low', 'open less than close',
       'central pivot point','to buy']]



data_raw.head(10)

,ema(4),ema(10),macd,macd 1st d,high low span,high above ema,close above ema,close relative to high,volume change,volume relative to span,3 days ago,2 days ago,1 day ago,close relative to low,open less than close,central pivot point,to buy
0,1.022028,1.218713,-0.196685,-1.232292,0.017941,-0.017329,-0.022028,0.004699,-0.422993,-0.042415,1,0,0,-0.013242,0,-0.002848,0
1,1.003295,0.969274,0.034020,0.230705,0.021643,0.002589,-0.003295,0.005884,0.152577,0.141851,0,0,0,-0.015760,0,-0.003292,0
2,1.008304,1.017578,-0.009274,-0.043294,0.022083,0.009851,-0.008304,0.018155,-0.385268,-0.057319,0,0,1,-0.003928,1,0.004742,0
3,1.011146,1.013491,-0.002345,0.006929,0.014908,-0.002245,-0.011146,0.008902,-0.143073,-0.104197,0,1,0,-0.006006,0,0.000965,0
4,1.010090,1.013327,-0.003237,-0.000892,0.017256,-0.002217,-0.010090,0.007873,0.178705,0.096563,1,0,0,-0.009383,0,-0.000503,0
5,1.005597,1.007042,-0.001445,0.001792,0.012717,0.004964,-0.005597,0.010561,-0.326795,-0.038914,0,0,0,-0.002155,0,0.002802,0
6,0.999289,0.998770,0.000519,0.001964,0.015628,0.002316,0.000711,0.001606,0.022276,0.701585,0,0,0,-0.014023,0,-0.004139,0
7,1.005471,1.007555,-0.002083,-0.002603,0.011999,0.005987,-0.005471,0.011458,-0.172976,-0.069366,0,0,1,-0.000540,0,0.003639,0
8,1.017774,1.023844,-0.006070,-0.003986,0.036639,0.009235,-0.017774,0.027009,0.623675,0.058748,0,1,0,-0.009630,1,0.005793,0
9,1.009450,1.011781,-0.002331,0.003739,0.018449,0.003254,-0.009450,0.012704,-0.719079,-0.025656,1,0,0,-0.005745,0,0.002320,1


In [4]:
# Clean the rows -- keep all rows where ema(10) != 0.00000000000000

data = data_raw.loc[data_raw['ema(10)'] != 0.000000,:]
data.count()


ema(4)                     19275
ema(10)                    19275
macd                       19275
macd 1st d                 19275
high low span              19275
high above ema             19275
close above ema            19275
close relative to high     19275
volume change              19275
volume relative to span    19275
3 days ago                 19275
2 days ago                 19275
1 day ago                  19275
close relative to low      19275
open less than close       19275
central pivot point        19275
to buy                     19275
dtype: int64

In [5]:
# Setup the Y target
target = data["to buy"]
target_names = ["Don't Buy", "Buy"]
target

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        1
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       1
24       0
25       0
26       0
27       0
28       0
29       0
        ..
19245    0
19246    0
19247    0
19248    0
19249    0
19250    0
19251    0
19252    0
19253    0
19254    0
19255    0
19256    0
19257    0
19258    0
19259    0
19260    0
19261    0
19262    0
19263    0
19264    0
19265    0
19266    0
19267    0
19268    0
19269    0
19270    0
19271    0
19272    0
19273    0
19274    0
Name: to buy, Length: 19275, dtype: int64

In [6]:
feature_names = data.columns
feature_names

Index(['ema(4)', 'ema(10)', 'macd', 'macd 1st d', 'high low span',
       'high above ema', 'close above ema', 'close relative to high',
       'volume change', 'volume relative to span', '3 days ago', '2 days ago',
       '1 day ago', 'close relative to low', 'open less than close',
       'central pivot point', 'to buy'],
      dtype='object')

In [7]:
# Remove the Y-axis column, to buy
data = data.drop('to buy', axis=1)


In [8]:
data

,ema(4),ema(10),macd,macd 1st d,high low span,high above ema,close above ema,close relative to high,volume change,volume relative to span,3 days ago,2 days ago,1 day ago,close relative to low,open less than close,central pivot point
0,1.022028,1.218713,-0.196685,-1.232292,0.017941,-0.017329,-0.022028,0.004699,-0.422993,-0.042415,1,0,0,-0.013242,0,-0.002848
1,1.003295,0.969274,0.034020,0.230705,0.021643,0.002589,-0.003295,0.005884,0.152577,0.141851,0,0,0,-0.015760,0,-0.003292
2,1.008304,1.017578,-0.009274,-0.043294,0.022083,0.009851,-0.008304,0.018155,-0.385268,-0.057319,0,0,1,-0.003928,1,0.004742
3,1.011146,1.013491,-0.002345,0.006929,0.014908,-0.002245,-0.011146,0.008902,-0.143073,-0.104197,0,1,0,-0.006006,0,0.000965
4,1.010090,1.013327,-0.003237,-0.000892,0.017256,-0.002217,-0.010090,0.007873,0.178705,0.096563,1,0,0,-0.009383,0,-0.000503
5,1.005597,1.007042,-0.001445,0.001792,0.012717,0.004964,-0.005597,0.010561,-0.326795,-0.038914,0,0,0,-0.002155,0,0.002802
6,0.999289,0.998770,0.000519,0.001964,0.015628,0.002316,0.000711,0.001606,0.022276,0.701585,0,0,0,-0.014023,0,-0.004139
7,1.005471,1.007555,-0.002083,-0.002603,0.011999,0.005987,-0.005471,0.011458,-0.172976,-0.069366,0,0,1,-0.000540,0,0.003639
8,1.017774,1.023844,-0.006070,-0.003986,0.036639,0.009235,-0.017774,0.027009,0.623675,0.058748,0,1,0,-0.009630,1,0.005793
9,1.009450,1.011781,-0.002331,0.003739,0.018449,0.003254,-0.009450,0.012704,-0.719079,-0.025656,1,0,0,-0.005745,0,0.002320


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
len(X_train), len(X_test)

(14456, 4819)

In [10]:
# Support vector machine linear classifier -- do all 3 kernals in a row
# Comment out to run rbf or poly kernels if needed
from sklearn.svm import SVC 
lin_model = SVC(kernel='linear')
rbf_model = SVC(kernel='rbf')
poly_model = SVC(kernel='poly')

In [ ]:
# Run the linear and non-linear kernals (rbf, poly).
# Comment out to run rbf or poly 
#svc_lin = lin_model.fit(X_train, y_train)
#svc_rbf = rbf_model.fit(X_train, y_train)
svc_poly = poly_model.fit(X_train, y_train)

C:\anaconda\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [1]:
# comment out to print rbf or poly 

print("Linear kernal Test Accuracy: %.3f" % svc_lin.score(X_test, y_test))

print("RBF kernal Test Accuracy: %.3f" % svc_rbf.score(X_test, y_test))
print("Poly kernal Test Accuracy: %.3f" % svc_poly.score(X_test, y_test))

NameError: name 'svc_lin' is not defined

In [ ]:
# Check the correlation matrix to see the autocorrelated variables and plot it out
# Will run the correlation matrix for the last kernel run

correlations = data.corr()
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,17,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
#ax.set_xticklabels(feature_names)
ax.set_yticklabels(feature_names)
plt.show()

In [ ]:
correlations

In [ ]:
''' Go ahead and Test the model with the other half of the data
from sklearn.metrics import classification_report
lin_predictions = svc_lin.predict(X_test)
print(classification_report(y_test, lin_predictions, target_names = target_names))

rbf_predictions = svc_rbf.predict(X_test)
print(classification_report(y_test, rbf_predictions, target_names = target_names))

poly_predictions = svc_poly.predict(X_test)
print(classification_report(y_test, poly_predictions, target_names = target_names))'''

In [ ]:
# Tune the hyper parameters as is
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1,5,10, 50, 75, 100, 150, 200, 250],
              'gamma': [0.0001,0.0005,0.001, 0.01]}
lin_grid = GridSearchCV(lin_model, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
lin_grid.fit(X_train, y_train)


In [ ]:
print(lin_grid.best_params_)
print(lin_grid)
#print(lin_grid.best_score_)
print('Best Score from grid search:',lin_grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
lin_predictions = lin_grid.predict(X_test)


results = confusion_matrix(y_test, lin_predictions) 
print ('Confusion Matrix :')
print(results)

print('Accuracy Score :',accuracy_score(y_test, lin_predictions)) 
print(classification_report(y_test, lin_predictions,
                            target_names=["down", "up"]))

In [ ]:
# Do the hyperparameter search for RBF
# Before doing this,  run the model for the first time above
#rbf_grid = GridSearchCV(rbf_model, param_grid, verbose=3)
#rbf_grid.fit(X_train,y_train)


In [ ]:
#print(rbf_grid.best_params_)
#print(rbf_grid)
#print(lin_grid.best_score_)
#print('Best Score from grid search:',rbf_grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model

#rbf_predictions = rbf_grid.predict(X_test)


#results = confusion_matrix(y_test, rbf_predictions) 
#print ('Confusion Matrix :')
#print(results)

#print('Accuracy Score :',accuracy_score(y_test, rbf_predictions)) 
#print(classification_report(y_test, rbf_predictions,
#                            target_names=["down", "up"]))

In [ ]:
# Do the hyperparameter search for Poly kernel
poly_grid = GridSearchCV(poly_model, param_grid, verbose=3)
poly_grid.fit(X_train,y_train)


In [ ]:
print(poly_grid.best_params_)
print(poly_grid)
#print(lin_grid.best_score_)
print('Best Score from grid search:',poly_grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model

poly_predictions = poly_grid.predict(X_test)


results = confusion_matrix(y_test, poly_predictions) 
print ('Confusion Matrix :')
print(results)

print('Accuracy Score :',accuracy_score(y_test, poly_predictions)) 
print(classification_report(y_test, poly_predictions,
                            target_names=["down", "up"]))

In [ ]:
print(lin_grid.best_score_)
print(rbf_grid.best_score_)
print(poly_grid.best_score_)

# The next section is option in that In is a parameter optimization section where  we remove the features with a strong correlation positive or negative correlation to another feature.   

# "Strong" is defined as a <-0.9 or >+0.9 correlation between two features.  Only one of the features is needed, so features can be dropped out.

# Very weak features may be dropped out as well (Close to 0 correlation).

# After dropping out redundant variables,  rerun the model to test for accuracy, recall, F-Factor etc. 

# Below the data is updated based on the AAPL 9 year data model. So, please looked at the correlation matrix above and see if any data can correlated before running this next section of code. 


In [ ]:
# Take out the positive duplicate correlations

data_update = data.drop("close above ema4", axis=1)
data_update = data_update.drop("SP close above ema", axis=1)
data_update = data_update.drop("DJ close above ema4", axis=1)
data_update = data_update.drop("DJ high low", axis=1)

# Also take out the strong negative duplicate correlations
data_update = data_update.drop("DJ close to high var %", axis=1)
data_update = data_update.drop("DJ CC delta", axis=1)

feature_up_names = data_update.columns
data_update.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_update, target, random_state=42)

In [ ]:
lin_model2 = SVC(kernel='linear')
rbf_model2 = SVC(kernel='rbf')
poly_model2 = SVC(kernel='poly')

In [ ]:
svc_lin2 = lin_model2.fit(X_train, y_train)
svc_rbf2 = rbf_model2.fit(X_train, y_train)
svc_poly2 = poly_model2.fit(X_train, y_train)

In [ ]:
print("Linear kernal 2 Test Accuracy: %.3f" % svc_lin2.score(X_test, y_test))
print("RBF kernal Test 2 Accuracy: %.3f" % svc_rbf2.score(X_test, y_test))
print("Poly kernal Test 2 Accuracy: %.3f" % svc_poly2.score(X_test, y_test))

In [ ]:
# check the correlation matrix to see the autocorrelated variables and plot it out

new_correlations = data_update.corr()
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(new_correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,16,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
#ax.set_xticklabels(feature_names)
ax.set_yticklabels(feature_up_names)
plt.show()

In [ ]:
new_correlations

# second interation of data removal
data_update2 = data_update.drop("DJ CC delta", axis=1)
feature_2_names = data_update2.columns
data_update2.head()

X_train, X_test, y_train, y_test = train_test_split(data_update2, target, random_state=7)

lin_model3 = SVC(kernel='linear')
rbf_model3 = SVC(kernel='rbf')
poly_model3 = SVC(kernel='poly')

svc_lin3 = lin_model3.fit(X_train, y_train)
svc_rbf3 = rbf_model3.fit(X_train, y_train)
svc_poly3 = poly_model3.fit(X_train, y_train)

print("Linear kernal Test Accuracy: %.3f" % svc_lin3.score(X_test, y_test))
print("RBF kernal Test Accuracy: %.3f" % svc_rbf3.score(X_test, y_test))
print("Poly kernal Test Accuracy: %.3f" % svc_poly3.score(X_test, y_test))

# check the correlation matrix to see the autocorrelated variables and plot it out

new_correlations_2 = data_update2.corr()
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(new_correlations_2, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,16,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
#ax.set_xticklabels(feature_names)
ax.set_yticklabels(feature_2_names)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
lin_predictions = svc_lin2.predict(X_test)
print(classification_report(y_test, lin_predictions, target_names = target_names))

rbf_predictions = svc_rbf2.predict(X_test)
print(classification_report(y_test, rbf_predictions, target_names = target_names))

poly_predictions = svc_poly2.predict(X_test)
print(classification_report(y_test, poly_predictions, target_names = target_names))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [10, 50, 75, 100, 150, 200, 250],
              'gamma': [0.0001,0.0005,0.001, 0.01]}
lin_grid = GridSearchCV(lin_model2, param_grid, verbose=3)


In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
lin_grid.fit(X_train, y_train)

In [ ]:
print(lin_grid.best_params_)

In [ ]:
print(lin_grid)
#print(lin_grid.best_score_)
print('Best Score from grid search:',lin_grid.best_score_)

In [ ]:
# Make predictions from new data file


In [ ]:
# Make predictions with the hypertuned model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
lin_predictions = lin_grid.predict(X_test)


results = confusion_matrix(y_test, lin_predictions) 
print ('Confusion Matrix :')
print(results)

print('Accuracy Score :',accuracy_score(y_test, lin_predictions)) 
print(classification_report(y_test, lin_predictions,
                            target_names=["down", "up"]))


In [ ]:
# Check the same analysis for RBF kernel

In [ ]:
rbf_grid = GridSearchCV(rbf_model2, param_grid, verbose=3)
rbf_grid.fit(X_train,y_train)

In [ ]:
print(rbf_grid.best_params_)
print(rbf_grid)
print(rbf_grid.best_score_)

In [ ]:
rbf_predictions = rbf_grid.predict(X_test)


results = confusion_matrix(y_test, lin_predictions) 
print ('Confusion Matrix :')
print(results)

print('Accuracy Score :',accuracy_score(y_test, rbf_predictions)) 
print(classification_report(y_test, rbf_predictions,
                            target_names=["down", "up"]))

In [ ]:
# Perform the same hyper-parameterization search for poly kernel

In [ ]:
poly_grid = GridSearchCV(poly_model2, param_grid, verbose=3)

poly_grid.fit(X_train,y_train)

In [ ]:
# List the best parameters for this dataset
print(poly_grid.best_params_)


In [ ]:
# List the best score


print(poly_grid)
print(poly_grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
#lin_predictions = lin_grid.predict(X_test)
#rbf_predictions = rbf_grid.predict(X_test)
#poly_predictions = poly_grid.predict(X_test)
poly_predictions = poly_grid.predict(X_test)


results = confusion_matrix(y_test, poly_predictions) 
print ('Confusion Matrix :')
print(results)

print('Accuracy Score :',accuracy_score(y_test, poly_predictions)) 
print(classification_report(y_test, poly_predictions,
                            target_names=["down", "up"]))

In [ ]:
# Summarize

print(classification_report(y_test, lin_predictions,
                            target_names=["down", "up"]))
print(classification_report(y_test, rbf_predictions,
                            target_names=["down", "up"]))
print(classification_report(y_test, poly_predictions,
                            target_names=["down", "up"]))

In [ ]:
print(lin_grid.best_score_)
print(rbf_grid.best_score_)
print(poly_grid.best_score_)

In [ ]:
pickle.dump(lin_grid, open('lin_model.pkl','wb'))

In [ ]:
# Prepare the model for prediction
lin_model = pickle.load(open('lin_model.pkl', 'rb'))


In [ ]:
# prepare the prediciton data
pred = pd.read_csv(os.path.join("Resources", "AAPL_0605_c_16_0519.csv"))
pred.head()

In [ ]:
actual_data= pred["price change"]


In [ ]:
pred = pred.drop("price change", axis=1)
names = pred.columns
pred.head()

In [ ]:
names

In [ ]:
pred = pred.drop("close above ema4", axis=1)
pred = pred.drop("SP close above ema", axis=1)
pred = pred.drop("DJ close above ema4", axis=1)
pred = pred.drop("DJ high low", axis=1)

# Also take out the strong negative duplicate correlations
pred = pred.drop("DJ close to high var %", axis=1)
pred = pred.drop("DJ CC delta", axis=1)

valid_names = pred.columns
pred.head()

In [ ]:
Xnew = []
Xnew = pred


In [ ]:
ynew = lin_model.predict(Xnew)

ynew, actual_data
    

In [ ]:
diff = ynew - actual_data
diff

In [ ]:
# Look at the results
lw = 2

svms = [rbf_model, lin_model, poly_model]
kernel_label = ['RBF', 'Linear', 'Polynomial']
model_color = ['m', 'c', 'g']

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10), sharey=True)
for ix, svm in enumerate(svms):
    axes[ix].plot(X, .fit(X, y).predict(X), color=model_color[ix], lw=lw,
                  label='{} model'.format(kernel_label[ix]))
    axes[ix].scatter(X[svr.support_], y[svr.support_], facecolor="none",
                     edgecolor=model_color[ix], s=50,
                     label='{} support vectors'.format(kernel_label[ix]))
    axes[ix].scatter(X[np.setdiff1d(np.arange(len(X)), svr.support_)],
                     y[np.setdiff1d(np.arange(len(X)), svr.support_)],
                     facecolor="none", edgecolor="k", s=50,
                     label='other training data')
    axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
                    ncol=1, fancybox=True, shadow=True)

fig.text(0.5, 0.04, 'data', ha='center', va='center')
fig.text(0.06, 0.5, 'target', ha='center', va='center', rotation='vertical')
fig.suptitle("Support Vector Machine", fontsize=14)
plt.show()


In [ ]:
# Look at the results
lw = 2

svrs = [svr_rbf, svr_lin, svr_poly]
kernel_label = ['RBF', 'Linear', 'Polynomial']
model_color = ['m', 'c', 'g']

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10), sharey=True)
for ix, svr in enumerate(svrs):
    axes[ix].plot(X, svr.fit(X, y).predict(X), color=model_color[ix], lw=lw,
                  label='{} model'.format(kernel_label[ix]))
    axes[ix].scatter(X[svr.support_], y[svr.support_], facecolor="none",
                     edgecolor=model_color[ix], s=50,
                     label='{} support vectors'.format(kernel_label[ix]))
    axes[ix].scatter(X[np.setdiff1d(np.arange(len(X)), svr.support_)],
                     y[np.setdiff1d(np.arange(len(X)), svr.support_)],
                     facecolor="none", edgecolor="k", s=50,
                     label='other training data')
    axes[ix].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
                    ncol=1, fancybox=True, shadow=True)

fig.text(0.5, 0.04, 'data', ha='center', va='center')
fig.text(0.06, 0.5, 'target', ha='center', va='center', rotation='vertical')
fig.suptitle("Support Vector Regression", fontsize=14)
plt.show()